This exercise will require you to pull some data from the Qunadl API. Qaundl is currently the most widely used aggregator of financial market data.

As a first step, you will need to register a free account on the http://www.quandl.com website.

After you register, you will be provided with a unique API key, that you should store:

In [12]:
# Store the API key as a string - according to PEP8, constants are always named in all upper case
API_KEY = 'APIKEY'

Qaundl has a large number of data sources, but, unfortunately, most of them require a Premium subscription. Still, there are also a good number of free datasets.

For this mini project, we will focus on equities data from the Frankfurt Stock Exhange (FSE), which is available for free. We'll try and analyze the stock prices of a company called Carl Zeiss Meditec, which manufactures tools for eye examinations, as well as medical lasers for laser eye surgery: https://www.zeiss.com/meditec/int/home.html. The company is listed under the stock ticker AFX_X.

You can find the detailed Quandl API instructions here: https://docs.quandl.com/docs/time-series

While there is a dedicated Python package for connecting to the Quandl API, we would prefer that you use the *requests* package, which can be easily downloaded using *pip* or *conda*. You can find the documentation for the package here: http://docs.python-requests.org/en/master/ 

Finally, apart from the *requests* package, you are encouraged to not use any third party Python packages, such as *pandas*, and instead focus on what's available in the Python Standard Library (the *collections* module might come in handy: https://pymotw.com/3/collections/ ).
Also, since you won't have access to DataFrames, you are encouraged to us Python's native data structures - preferably dictionaries, though some questions can also be answered using lists.
You can read more on these data structures here: https://docs.python.org/3/tutorial/datastructures.html

Keep in mind that the JSON responses you will be getting from the API map almost one-to-one to Python's dictionaries. Unfortunately, they can be very nested, so make sure you read up on indexing dictionaries in the documentation provided above.

These are your tasks for this mini project:

1. Collect data from the Franfurt Stock Exchange, for the ticker AFX_X, for the whole year 2017 (keep in mind that the date format is YYYY-MM-DD).
2. Convert the returned JSON object into a Python dictionary.
3. Calculate what the highest and lowest opening prices were for the stock in this period.
4. What was the largest change in any one day (based on High and Low price)?
5. What was the largest change between any two days (based on Closing Price)?
6. What was the average daily trading volume during this year?
7. (Optional) What was the median trading volume during this year. (Note: you may need to implement your own function for calculating the median.)

In [2]:
import requests

In [13]:
# Task 1 
url = 'https://www.quandl.com/api/v3/datasets/FSE/AFX_X.json?api_key=APIKEY&start_date=2017-01-01&end_date=2017-12-31'
params = dict(key=API_KEY)
data = requests.get(url, params= params)

# Task 2
data = data.json()

#print(data)

In [4]:
print(type(data))

<class 'dict'>


In [5]:
# explore column info and first few rows

columns = data['dataset']['column_names']
rows = data['dataset']['data']

print(columns)
print(" ")
print(rows[:5])

['Date', 'Open', 'High', 'Low', 'Close', 'Change', 'Traded Volume', 'Turnover', 'Last Price of the Day', 'Daily Traded Units', 'Daily Turnover']
 
[['2017-12-29', 51.76, 51.94, 51.45, 51.76, None, 34640.0, 1792304.0, None, None, None], ['2017-12-28', 51.65, 51.82, 51.43, 51.6, None, 40660.0, 2099024.0, None, None, None], ['2017-12-27', 51.45, 51.89, 50.76, 51.82, None, 57452.0, 2957018.0, None, None, None], ['2017-12-22', 51.05, 51.5, 50.92, 51.32, None, 71165.0, 3641949.0, None, None, None], ['2017-12-21', 51.16, 51.52, 50.9, 51.4, None, 120649.0, 6179433.0, None, None, None]]


In [6]:
# Task 3
# find highest and lowest opening prices

opening_high = 0.00
opening_low = 100.00
for row in rows:
    opening_price = row[1]
    try:
        if opening_price > opening_high:
            opening_high = opening_price
        if opening_price < opening_low:
            opening_low = opening_price
    except:
        continue
print("The Opening high price is $" + str(opening_high) + ", and the Opening low price is $" + str(opening_low) + ".")

The Opening high price is $53.11, and the Opening low price is $34.0.


In [7]:
# Task 4
# find greatest change in one day

max_diff = 0
for row in rows:
    high = row[2]
    low = row[3]
    try:
        diff = abs(high-low)
        if diff > max_diff:
            max_diff = diff
            day = row[0]
    except:
        continue
        
max_diff = round(max_diff,2)

    
print("The greatest difference in High and Low prices reported in one day was on " + day + " with a recorded difference of $" + str(max_diff) + " dollars.") 

The greatest difference in High and Low prices reported in one day was on 2017-05-11 with a recorded difference of $2.81 dollars.


In [8]:
# Task 5
# find greatest change between two days, as reported in the Close column

max_diff = 0
yest_close = 0
for row in rows:
    close = row[4]
    diff = abs(close - yest_close)
    try:
        diff = abs(close - yest_close)
        if diff > max_diff:
            max_diff = diff
            day = row[0]
    except:
        continue
        
max_diff = round(max_diff,2)

    
print("The greatest difference in closing prices between two days was recorded on " + day + " with a difference of $" + str(max_diff) + " dollars.") 

The greatest difference in closing prices between two days was recorded on 2017-12-13 with a difference of $53.09 dollars.


In [9]:
# Task 6
# find average trading value 

total_vol = 0
count = 0 
for row in rows:
    vol = row[6]
    try:
        total_vol = total_vol + vol
        count += 1
    except:
        print("Error with row: " + row)

avg_vol = round((total_vol / count),2)
print("The average daily trading volume during the year is $" + str(avg_vol) + ".")

The average daily trading volume during the year is $89124.34.


In [10]:
# Task 7
# find median trading value 

all_vols = list()
for row in rows:
    vol = row[6]
    all_vols.append(vol)

    
all_vols = sorted(all_vols)

if len(all_vols) % 2 != 0:
    indx = int((len(all_vols) + 1) / 2)
    median = all_vols[indx]
else:
    indx1 = int(len(all_vols) / 2)
    indx2 = int(indx1 + 1)
    median = round(((indx1 + indx2) / 2),2)
      

print("The median trading volume for the year is $" + str(median) + ".")

The median trading volume for the year is $76600.0.
